In [1]:
import sys
sys.path.insert(0, '/Users/shiahlints/galvanize/FIXGITPROBLEM/booking-agent-ai')
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import find_nearest_venues as fnv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
'''import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator'''


'import pyspark\nfrom pyspark.sql.types import *\nfrom pyspark.ml.tuning import TrainValidationSplit\nfrom pyspark.ml.recommendation import ALS\nfrom pyspark.ml.evaluation import RegressionEvaluator'

In [2]:
train_events = pd.read_pickle('../data/train_events_df.p')
test_events = pd.read_pickle('../data/test_events_raw_df.p')
train_venues = pd.read_pickle('../data/train_venues_df.p')
artists = pd.read_pickle('../data/master_artists_df.p')


# We start by sampling form the test events
we will use the venue id to get the nearest 100 venues to where this 
event took place. We will then check if the model return this venues 
as one of the top ten recomendations. The model is trained on the train events.
The data has been split in such a way that there is no leakage.

In [12]:
number_of_samples = 50
samples = test_events.sample(number_of_samples)

# Calibrate the vunue finder
The venue finder is calibrated with all the venues
in the training set. This was it will return venues only in the training set.

In [6]:
vf = fnv.Venue_Finder(train_venues)

# Make a dataset to train the model on 
and filter out venues without bios

In [7]:
data = train_events[['artist_id', 'venue_bio', 'venue_id']]
data = data[~data.venue_bio.isnull()]

# Train a count vectorizer and a MultinomialNb

In [8]:
vectorizer = CountVectorizer(stop_words = 'english', max_features = 1000)
counts = vectorizer.fit_transform(data['venue_bio'].values)
classifier = MultinomialNB()
targets = data['venue_id'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer(stop_words = 'english', max_features = 1000)),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(data['venue_bio'].values, data['venue_id'].values)

ratings1 = pipeline.predict_proba(['this is a test'])

# Test Model

In [18]:
number_of_samples = 50
samples = test_events.sample(number_of_samples)

In [21]:

new_venue = 0 
model_sucesses = 0
model_failures = 0
number_venues_to_search = 100
top_n_venues = 10
no_bio = 0
for i, event in samples.iterrows():
    artist_id = event.artist_id
    venue_id = event.venue_id
    artist_bio = event.artist_bio
    if venue_id not in train_venues.venue_id.values:
        new_venue += 1
        continue
    
    if artist_bio == None:
        no_bio += 1
        continue
    else:
        # First get a array of the classes. Predict probab only returns the probibility of 
        # document "bio" belonging to a class "venue". We are only interested in local 
        #venues so we need this array to filter the probibilities:
        all_venues = classifier.classes_
        
        # get_nearest_venues_by_venue returns a data_frame contianing all the venue info
        # we are only interested in the venue_id values
        nearest_venues = (vf.get_nearest_venues_by_venue(venue_id, number_venues_to_search).
                          venue_id.values)
        
        # these next to lines return a array of probabilities that relate to the 
        # classes in all_venues
        bio_vectorized = vectorizer.transform([artist_bio])
        ratings = classifier.predict_proba(bio_vectorized).reshape(-1,)
        
        #now we make a filter to get only the venues close by 
        venue_filter = np.isin(all_venues, nearest_venues)
        
        #and apply it on the venues and porbs
        local_venues = all_venues[venue_filter]
        local_probs = ratings[venue_filter]
        
        #Now make a sorter with the provs to sort the venues from most to least likely
        sorter = local_probs.argsort()
        top_local_venues_sorted = local_venues[sorter[:number_venues_to_search-10:-1]]
    
    
        if venue_id in top_local_venues_sorted:
            model_sucesses += 1
        else:
            model_failures += 1

print(model_sucesses/(model_failures+model_sucesses))



0.35294117647058826


Thats better then .1 lest wright that all into a function that so we can test. 

In [26]:
def top_ten_rate(samples, classifier, vectorizer):
    new_venue = 0 
    model_sucesses = 0
    model_failures = 0
    number_venues_to_search = 100
    top_n_venues = 10
    no_bio = 0
    for i, event in samples.iterrows():
        artist_id = event.artist_id
        venue_id = event.venue_id
        artist_bio = event.artist_bio
        if venue_id not in train_venues.venue_id.values:
            new_venue += 1
            continue

        if artist_bio == None:
            no_bio += 1
            continue
        else:
            # First get a array of the classes. Predict probab only returns the probibility of 
            # document "bio" belonging to a class "venue". We are only interested in local 
            #venues so we need this array to filter the probibilities:
            all_venues = classifier.classes_

            # get_nearest_venues_by_venue returns a data_frame contianing all the venue info
            # we are only interested in the venue_id values
            nearest_venues = (vf.get_nearest_venues_by_venue(venue_id, number_venues_to_search).
                              venue_id.values)

            # these next to lines return a array of probabilities that relate to the 
            # classes in all_venues
            bio_vectorized = vectorizer.transform([artist_bio])
            ratings = classifier.predict_proba(bio_vectorized).reshape(-1,)

            #now we make a filter to get only the venues close by 
            venue_filter = np.isin(all_venues, nearest_venues)

            #and apply it on the venues and porbs
            local_venues = all_venues[venue_filter]
            local_probs = ratings[venue_filter]

            #Now make a sorter with the provs to sort the venues from most to least likely
            sorter = local_probs.argsort()
            top_local_venues_sorted = local_venues[sorter[:number_venues_to_search-10:-1]]


            if venue_id in top_local_venues_sorted:
                model_sucesses += 1
            else:
                model_failures += 1

    return(model_sucesses/(model_failures+model_sucesses))

In [27]:
results = []
for i in range(10):
    samples = test_events.sample(50)
    results.append(top_ten_rate(samples, classifier, vectorizer))
results   

[0.20833333333333334,
 0.2916666666666667,
 0.21739130434782608,
 0.18518518518518517,
 0.35,
 0.15789473684210525,
 0.2631578947368421,
 0.2857142857142857,
 0.19047619047619047,
 0.36363636363636365]

Thats a large deviation. 
lets take the mean of that and see what happens when we change the rand of the model

In [30]:
def build_model(features):
    vectorizer = CountVectorizer(stop_words = 'english', max_features = features)
    counts = vectorizer.fit_transform(data['venue_bio'].values)
    classifier = MultinomialNB()
    targets = data['venue_id'].values
    classifier.fit(counts, targets)
    return vectorizer, classifier 

In [38]:
def test_model(vectorizer, classifier):
    results = []
    for i in range(10):
        samples = test_events.sample(50)
        results.append(top_ten_rate(samples, classifier, vectorizer))
    print(results)
    return np.mean(results) 

In [46]:
features_list = [200, 400, 600, 800, 1000]
results = []
for features in features_list:
    v, c = build_model(features)
    results.append(test_model(v,c))
results

[0.1, 0.2608695652173913, 0.2631578947368421, 0.2857142857142857, 0.5238095238095238, 0.13043478260869565, 0.3157894736842105, 0.25, 0.25, 0.2608695652173913]
[0.18518518518518517, 0.17391304347826086, 0.27586206896551724, 0.34615384615384615, 0.27586206896551724, 0.2, 0.2777777777777778, 0.2, 0.32, 0.21052631578947367]
[0.3157894736842105, 0.2, 0.2916666666666667, 0.09523809523809523, 0.25, 0.2962962962962963, 0.125, 0.4166666666666667, 0.3076923076923077, 0.3333333333333333]
[0.20833333333333334, 0.10526315789473684, 0.125, 0.2777777777777778, 0.26666666666666666, 0.20833333333333334, 0.17391304347826086, 0.14285714285714285, 0.3333333333333333, 0.2]
[0.28, 0.3333333333333333, 0.2, 0.25, 0.16, 0.2, 0.20689655172413793, 0.19047619047619047, 0.2916666666666667, 0.21052631578947367]


[0.26406450909883405,
 0.24652803063155782,
 0.26316828395775765,
 0.20414777886745847,
 0.23228990579898018]

This is a decent result but other aproaches are better so lets drop this.
